# MaxCut and QAOA

Idea: want to understand a quantum algorithm that's useful now.

I started looking at the IBM lesson:
https://quantum.cloud.ibm.com/learning/en/courses/quantum-computing-in-practice/utility-scale-qaoa

I got confused by some of the explanations there, so did other reading and my own calculations.  

This is a summary of some things I've worked out, and things I haven't.


## MaxCut

The problem is we start with a graph and want to find a maximum cut: this is a bipartite subgraph with maximum number of edges.

So we have a graph $G=(V,E)$ and want to partition the vertices into two disjoint sets:
$$V = V_0 \cup V_1 $$

Example given: $V=\{0,1,2,3,4\}$ with edges $01, 02, 04, 12, 23, 34$.  Visually, we have a "02340" square attached to a "0120" triangle at the edge $02$.

This is small enough that you can check all possibilities by hand.  The four maximal solutions are:
$$ V_0 = \{ 0, 3\}, \;\; V_1 = \{ 1,2,4\} $$
$$ V_0 = \{ 2, 4\}, \;\; V_1 = \{ 0,1,3\} $$
$$ V_0 = \{ 1,2,4\}, \;\; V_1 = \{ 0,3\}$$
$$ V_0 = \{ 0,1,3\}, \;\; V_1 = \{ 2,4\} $$

Note that "up to symmetries" there's only one solution: choose a vertex in both the square and the triangle, and put that in a 2-element set with the opposite corner of the square.  There's two choices for the first vertex, and we can swap $V_0$ and $V_1$, so this gives four solutions.

Encode these partitions as basis states of a 5-qubit system, where qubit $i$ has state $k$ if $i\in V_k$.  So, using Qiskit notation where our qubits are ordered as $q_4q_3q_2q_1q_0$, we get the four solutions:
$$ 10110, \;\; 01011, \;\; 01001, \;\; 10100 $$







## Formulate as minimisation problem

Let $x_i$ be the state as before, so $x_i=0$ if $i\in V_0$, and $x_i=1$ if $i\in V_1$.

If we have an edge between $i$ and $j$, consider the following quantity:
$$ x_i + x_j - 2x_ix_j $$
Note that it's $0$ if $x_i$ and $x_j$ have the same state, and $1$ if they have different states.

So we want to maximise the sum of these terms over the edges.  Let $n=$ number of vertices.

Change to a minimisation problem by taking the negative of this quantity.  So we want to find:
$$ \min_{x\in\{0,1\}^n}  \sum_{ij\in E} \left( 2\,x_ix_j  - x_i - x_j \right) $$

We want to rewrite this as $x^TQx$ where $x$ is the column vector of $x_i$'s.  The lesson doesn't explain how to do this, but you can check that the following matrix works:
$$ Q_{ii}=-\text{val}(i) \;\; \text{ and for $i\neq j$,} \;\; Q_{ij} = \begin{cases}
+1 &ij\in E\\
0  &ij\notin E
\end{cases}$$
Here $\text{val}(i)=\{j : ij\in E\}$ is the valence of the vertex $i$.  We used that $x_i^2=x_i$ as $x_i\in\{0,1\}$.

So we want to minimise $x^T Q x$.

Next do a change of variables (insipred by Ising model??)
$$ x_i = \frac{1-z_i}2 \;\; \Leftrightarrow \;\; z_i = 1-2x_i $$
Note this means:
$$ x=0 \Leftrightarrow z=+1 \;\;\;\; \text{ and } \;\;\;\; x=1 \Leftrightarrow z=-1 $$
Note that
$$ 4x^TQx %= 1^TQ1 - z^T Q1 - 1^tQz + z^TQz 
           = z^TQz - 1^t(Q+Q^t)z + 1^TQ1$$
where $1,z$ are the column vectors of $1$'s and $z_i$'s, respectively.

As we've fixed $Q$ and are minimising over $z$, we can ignore the summand $1^TQ1$ (though one can check from the definition of $Q$ that this is zero anyway).  Also from def of $Q$ we get that $1^t(Q+Q^t)$ is a zero vector, so 
$$ 4x^TQx            = z^TQz$$
and we can just minimise $z^TQz$.



## Make it quantum

Now we'll change interpretation.  I don't understand how we're meant to get from what's above to what's below, but I can see that both work.

Let $Z_iZ_j$ denote applying $Z$ in the $i$ and $j$ qubit positions.  Recall Qubit goes right to left for counting, so with 5 qubits we have
$$ Z_0Z_1 = I\otimes I\otimes I\otimes Z\otimes Z$$

The lesson says $H_C=\sum_{ij\in E}Q_{ij}\, Z_iZ_j$, but as $Q_{ij}$ is always $+1$ this seems to just be
$$H_C=\sum_{ij\in E} Z_iZ_j$$
This is called the "cost function Hamiltonian".

Note that it's not unitary (except in very special cases). 

Example: for the line graph 0 -- 1 -- 2, we want to find $010$ or $101$.  These turn out to be eigenvectors with e-value $-2$ for $H_C=IZZ+ZZI$ ("ground states").  Note every basis state is an eigenvector.  $000$ and $111$ have e-value $+2$; everything else has e-value $0$.

Aside: when we get further this seems not to work.  Alternative seen elsewhere:
$$ H_C = \frac12 \sum_{ij\in E}(I-Z_iZ_j)$$
Justificiation (this is vague): $I-Z_iZ_j$ will have eigenvalue $0$ if the states agree and $1$ if they don't, so we pick up a phase of $e^{-i\gamma}$ (see below) for edges that appear in the cut, and these should have good interference (??)


To get a unitary matrix, we will exponentiate.  Note that $Z=\text{diag}(1,-1)$ is a diagonal matrix, so we can exponentiate by simply applying $\exp$ to the diagonal entries. 

The circuit involves a "mixer Hamiltonian", which "is the simple, sum of Pauli-X operations on each node".  If I'm understanding that correctly, then for a graph with two vertices this would be $X\otimes I + I\otimes X$.  This isn't diagonal, but there are tricks to exponentiate it (later).

Now what's the circuit?

 - First apply a Hadamard gate to every qubit, so we get a scaled sum of the $2^n$ possible basis states.
 - Then we repeat the following, each being a "layer":
   - First apply $exp(-i\gamma H_C)$, a unitary coming from the cost function Hamiltonian with parameter angle $\gamma$
   - Then apply $exp(-i\beta H_M)$, a unitary coming from the mixing Hamiltonian with parameter angle $\beta$
 - Note we can change the parameters, using $(\gamma_1,\beta_1)$ in the first layer, etc.
 - Finally, measure everything

Then if everything has worked, we should have higher probabilities of measuring a joint state which gives a maximal cut.

Note: the $\gamma$s and $\beta$s need to be chosen carefully (I don't understand how yet, but can see explicitly that some work better than others).

Note: the reason we use layers is to do with the "Adiabatic Theorem" on time-dependent Hamiltonians.  Compare with Trotter-Suzuki decomposition formula:
$$ e^{A+B} \approx (e^{A/n}e^{B/n})^n$$

## Implementing this.

Key observation:

the unitary
$$ e^{-\gamma Z\otimes Z} $$ breaks up as the following sequence of unitaries:

 - controlled not, with vertex 0 as control
 - rotation in Z axis of angle $-2\gamma$ (check $\pm$ sign issue!)
 - controlled not, with vertex 0 as control

Also note that rotation in Z axis can be given as a phase change constant times a change of phase for |1>.


# More to do!

